# All the results and visualizations for the Sampta Paper

In [1]:
#Import everything
from Main.test_manifold_algorithms import test_manifold_algorithms as tma
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from Helpers.regression_helpers import *

## Checking if we have our data run

### Regression

In [6]:
reg_df = read_json_files_to_dataframe("/yunity/arusty/Graph-Manifold-Alignment/Results/GRAE_Data")
print(f"File Length: {len(reg_df)}")
reg_df.sample(5)

File Length: 1459


,method,csv_file,split,Percent_of_Anchors,Best_Params,CE,FOSCTTM,Random Forest OOB,Random Forest Emb A,Random Forest Emb B,Nearest Neighbor A,Nearest Neighbor B,Nearest Neighbor (F1 score or RMSE) A,Nearest Neighbor (F1 score or RMSE) B,Random Forest (F1 score or RMSE) A,Random Forest (F1 score or RMSE) B,GRAE,Parameter STD
147,SSMA,EnergyEfficiency,random,0.3,"{'knn': 18, 'Uincluded': False, 'Dincluded': T...","{'42': 0.7065201596052944, '1738': 0.667220901...","{'42': 0.351049794980743, '1738': 0.3482114529...","{'42': 0.5443334893245988, '1738': 0.390287662...","{'42': 0.47470714936564473, '1738': 0.43448264...","{'42': 0.5526382789120117, '1738': 0.594456437...","{'42': 0.7488455683989774, '1738': 0.592510261...","{'42': 0.7232195471490697, '1738': 0.620080665...","{'42': 0.5217216548748256, '1738': 0.635498960...","{'42': 0.5476916857017848, '1738': 0.613623811...","{'42': 0.7545172710899214, '1738': 0.748651111...","{'42': 0.696302333496144, '1738': 0.6339795685...","{'42': {'RF-OOB': 0.48488278821636044, 'KNN_A'...",{}
712,SPUD,hepatitis,skewed,0.3,"{'knn': 29, 'OD_method': 'default', 'agg_metho...",{'42': 0.6956521739130435},{'42': 0.007088846880907373},{'42': 0.6363636363636364},{'42': 0.5357142857142857},{'42': 0.5357142857142857},{'42': 0.6071428571428571},{'42': 0.6428571428571429},{'42': 0.6149068322981367},{'42': 0.6428571428571429},{'42': 0.5401360544217687},{'42': 0.5447004608294932},"{'42': {'RF-OOB': 0.65, 'KNN_A': 0.42857142857...","{'knn': 0.039550490894378044, 'OD_method': 0.0..."
505,NAMA,ecoli_5,distort,0.3,"{'OD_method': 'absolute_distance', 'agg_method...","{'42': 0.5626911314984709, '1738': 0.559633027...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': 0.8199233716475096, '1738': 0.821839080...","{'42': 0.8787878787878788, '1738': 0.878787878...","{'42': 0.9393939393939394, '1738': 0.803030303...","{'42': 0.8484848484848485, '1738': 0.848484848...","{'42': 0.8636363636363636, '1738': 0.803030303...","{'42': 0.8166469893742621, '1738': 0.842917675...","{'42': 0.8524299673756195, '1738': 0.772645372...","{'42': 0.8771186254380141, '1738': 0.879600437...","{'42': 0.9372561519826597, '1738': 0.799119399...","{'42': {'RF-OOB': 0.8026819923371648, 'KNN_A':...","{'OD_method': 0.003058103975535187, 'agg_metho..."
196,RF-NAMA,ionosphere,turn,0.3,"{'OD_method': 'mean', 'agg_method': 'log'}","{'42': 0.8888888888888888, '1738': 0.868945868...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': 0.9267857142857143, '1738': 0.9375, '52...","{'42': 0.39436619718309857, '1738': 0.32394366...","{'42': 0.39436619718309857, '1738': 0.32394366...","{'42': 0.39436619718309857, '1738': 0.32394366...","{'42': 0.39436619718309857, '1738': 0.32394366...","{'42': 0.22307582870963152, '1738': 0.15852562...","{'42': 0.22307582870963152, '1738': 0.15852562...","{'42': 0.22307582870963152, '1738': 0.15852562...","{'42': 0.22307582870963152, '1738': 0.15852562...","{'42': {'RF-OOB': 0.9303571428571429, 'KNN_A':...",{}
711,RF-NAMA,seeds,distort,0.3,"{'OD_method': 'mean', 'agg_method': 'log'}","{'42': 0.4472361809045226, '1738': 0.522613065...","{'42': 0.0, '1738': 0.0, '5271': 0.0, '9209': ...","{'42': 0.9339622641509434, '1738': 0.918238993...","{'42': 0.25, '1738': 0.2, '5271': 0.425, '9209...","{'42': 0.325, '1738': 0.2, '5271': 0.475, '920...","{'42': 0.225, '1738': 0.275, '5271': 0.35, '92...","{'42': 0.275, '1738': 0.225, '5271': 0.45, '92...","{'42': 0.15602678571428572, '1738': 0.19611111...","{'42': 0.19445043103448276, '1738': 0.11277777...","{'42': 0.16258741258741258, '1738': 0.06666666...","{'42': 0.299048178613396, '1738': 0.0666666666...","{'42': {'RF-OOB': 0.949685534591195, 'KNN_A': ...",{}


In [7]:
#This helps me to see where my tests have failed
proc_reg_df = reg_df.drop(columns = ["Best_Params", "Parameter STD"])

#Average out the random seeds
for columns in ["CE", "FOSCTTM", "Random Forest OOB", "Random Forest Emb", "Nearest Neighbor", "Nearest Neighbor (F1 score or RMSE)", "Random Forest (F1 score or RMSE)"]:
    proc_reg_df[columns] = [np.mean(list(d.values())) for d in proc_reg_df[columns].values]

proc_reg_df["Combined_Metric"] = proc_reg_df["CE"] - proc_reg_df["FOSCTTM"]

baselines = pd.read_csv("/yunity/arusty/Graph-Manifold-Alignment/Results/RegressionData/BaselineDataTrainTest.csv", index_col=0).rename(columns = {"KNN_BL_A": "A_Classification_Score", "KNN_BL_B": "B_Classification_Score"})
baselines_grouped = baselines.groupby(by = ["csv_file", "split"]).max().drop(columns = ["KNN"]).reset_index()

# Group by csv_file and select the row with the 3rd smallest knn value
third_smallest_knn = baselines.groupby(by = ["csv_file", "split"]).apply(lambda x: x.nsmallest(3, 'KNN').iloc[-1]).reset_index(drop=True)

# Ensure the resulting DataFrame has the same format as baselines_grouped
third_smallest_knn_grouped = third_smallest_knn.groupby(by=["csv_file", "split"]).agg({
    "A_Classification_Score": "max",
    "B_Classification_Score": "max"
}).reset_index()
proc_reg_df = proc_reg_df.merge(third_smallest_knn_grouped, how = "left")


reg_results_df = plt_methods_by_CSV_max(df = proc_reg_df, metric = "Combined_Metric", return_df=True)#.head(13) #, split = "skewed"
reg_results_df

KeyError: 'Random Forest Emb'

### Classification

In [ ]:
class_df = read_json_files_to_dataframe("/yunity/arusty/Graph-Manifold-Alignment/Results/ManifoldData")
print(f"File Length: {len(class_df)}")
class_df.sample(5)

File Length: 1524


,method,csv_file,split,Percent_of_Anchors,Best_Params,CE,FOSCTTM,Random Forest OOB,Random Forest Emb,Nearest Neighbor,Nearest Neighbor (F1 score or RMSE),Random Forest (F1 score or RMSE),GRAE,Parameter STD
275,MASH-,tic-tac-toe,skewed,0.3,"{'knn': 2, 'page_rank': 'full', 'DTM': 'log', ...",{'42': 0.6816283924843424},{'42': 0.17763008790931},{'42': 0.7976501305483029},{'42': 0.7708333333333334},{'42': 0.6848958333333334},{'42': 0.632517369117873},{'42': 0.7628759050790127},"{'42': {'RF-OOB': 0.77088772845953, 'KNN': 0.6...","{'knn': 0.07520889136205879, 'page_rank': 0.01..."
901,RF-SPUD,audiology,random,0.3,"{'knn': 6, 'OD_method': 'default', 'agg_method...","{'42': 0.4473684210526316, '1738': 0.547368421...","{'42': 0.00022160664819944597, '1738': 0.00016...","{'42': 0.3684210526315789, '1738': 0.552631578...","{'42': 0.3026315789473684, '1738': 0.578947368...","{'42': 0.2631578947368421, '1738': 0.552631578...","{'42': 0.21326987681970885, '1738': 0.50611269...","{'42': 0.25194430088525077, '1738': 0.49652123...","{'42': {'RF-OOB': 0.5164473684210527, 'KNN': 0...",{}
331,MASH,flare1,skewed,0.3,"{'knn': 23, 'page_rank': 'full', 'DTM': 'log',...",{'42': 0.8885448916408669},{'42': 0.16409147983782074},{'42': 0.877906976744186},{'42': 0.8769230769230769},{'42': 0.8769230769230769},{'42': 0.8194199243379572},{'42': 0.8324062301335029},"{'42': {'RF-OOB': 0.8798449612403101, 'KNN': 0...","{'connection_limit': 0.0, 'threshold': 0.0, 'e..."
621,RF-MASH,ecoli_5,distort,0.3,"{'knn': 23, 'page_rank': 'None', 'DTM': 'log',...","{'42': 0.7920489296636085, '1738': 0.804281345...","{'42': 0.016034003871727966, '1738': 0.0160340...","{'42': 0.8160919540229885, '1738': 0.831417624...","{'42': 0.8333333333333334, '1738': 0.848484848...","{'42': 0.8181818181818182, '1738': 0.818181818...","{'42': 0.8194841958843823, '1738': 0.820487911...","{'42': 0.8231001223676527, '1738': 0.840602581...","{'42': {'RF-OOB': 0.8831417624521073, 'KNN': 0...","{'connection_limit': 0.0, 'threshold': 0.0, 'e..."
1084,RF-MASH-,seeds,turn,0.3,"{'knn': 30, 'page_rank': 'None', 'DTM': 'log',...","{'42': 0.9899497487437185, '1738': 0.994974874...","{'42': 0.07585667028610388, '1738': 0.08121007...","{'42': 0.9654088050314465, '1738': 0.949685534...","{'42': 0.8875, '1738': 0.8875, '5271': 0.8625,...","{'42': 0.9, '1738': 0.8625, '5271': 0.875, '92...","{'42': 0.9008095238095237, '1738': 0.873440383...","{'42': 0.8878095238095238, '1738': 0.895581687...","{'42': {'RF-OOB': 0.9874213836477987, 'KNN': 0...","{'knn': 0.011418354131404433, 'page_rank': 0.0..."


In [ ]:
#This helps me to see where my tests have failed
proc_class_df = class_df.drop(columns = ["Best_Params", "Parameter STD"])

#Average out the random seeds
for columns in ["CE", "FOSCTTM", "Random Forest OOB", "Random Forest Emb", "Nearest Neighbor", "Nearest Neighbor (F1 score or RMSE)", "Random Forest (F1 score or RMSE)"]:
    proc_class_df[columns] = [np.mean(list(d.values())) for d in proc_class_df[columns].values]

proc_class_df["Combined_Metric"] = proc_class_df["CE"] - proc_class_df["FOSCTTM"]

baselines = pd.read_csv("/yunity/arusty/Graph-Manifold-Alignment/Results/ManifoldData/Data_DataFrame.csv", keep_default_na=False, na_values=['', 'NaN'], index_col= None
                          )[['csv_file', 'split', 'KNN', 'A_Classification_Score',
                            'B_Classification_Score']]

baselines_grouped = baselines.groupby(by = ["csv_file", "split"]).max().drop(columns = ["KNN"]).reset_index()

# Group by csv_file and select the row with the 3rd smallest knn value
third_smallest_knn = baselines.groupby(by = ["csv_file", "split"]).apply(lambda x: x.nsmallest(3, 'KNN').iloc[-1]).reset_index(drop=True)

# Ensure the resulting DataFrame has the same format as baselines_grouped
third_smallest_knn_grouped = third_smallest_knn.groupby(by=["csv_file", "split"]).agg({
    "A_Classification_Score": "max",
    "B_Classification_Score": "max"
}).reset_index()
proc_class_df = proc_class_df.merge(third_smallest_knn_grouped, how = "left")


cls_results_df = plt_methods_by_CSV_max(df = proc_class_df, metric = "Combined_Metric", return_df=True)#.head(13) #, split = "skewed"
cls_results_df

,csv_file,SSMA,MAGAN,DTA,SPUD,MASH,MASH-,RF-MASH-,NAMA,RF-NAMA,PCR,JLMA,MASH_RF,MALI_RF,MALI,SPUD_RF,BL_A,BL_B
0,zoo,NaN,NaN,NaN,0.207999,0.182413,0.187864,0.022380,0.158122,0.098559,NaN,NaN,0.222214,NaN,NaN,0.221782,0.415842,0.366337
1,water_potability,NaN,NaN,NaN,0.128309,0.249229,0.246931,-0.310442,0.002486,0.016896,NaN,NaN,0.252443,NaN,NaN,0.016111,0.294878,0.260567
2,winequality-red,NaN,NaN,NaN,0.106834,0.454108,0.447723,0.047582,0.026507,0.046990,NaN,NaN,0.370604,NaN,NaN,0.070614,0.315197,0.273296
3,b,NaN,NaN,NaN,1.000000,0.623400,0.454989,0.483450,0.870000,0.686667,NaN,NaN,0.558706,NaN,NaN,0.863333,NaN,NaN
4,hill_valley,NaN,NaN,NaN,0.546205,0.696134,0.691128,0.728979,0.570952,0.539601,NaN,NaN,0.696134,NaN,NaN,0.684565,0.790429,0.790429
5,heart_disease,NaN,NaN,NaN,0.626263,0.717172,0.717172,0.558401,0.515140,0.606049,NaN,NaN,0.701010,NaN,NaN,0.666653,0.757576,0.757576
6,Medicaldataset,NaN,NaN,NaN,0.613683,0.752287,0.755764,0.657278,0.561031,0.806822,NaN,NaN,0.750689,NaN,NaN,0.960425,0.717968,0.674754
7,audiology,NaN,NaN,NaN,0.714571,0.775654,0.753457,0.716939,0.504321,0.326316,NaN,NaN,0.755416,NaN,NaN,0.774715,0.900000,0.889474
8,heart_failure,NaN,NaN,NaN,0.749164,0.779698,0.770797,0.714860,0.658863,0.750502,NaN,NaN,0.736455,NaN,NaN,0.891639,0.799331,0.799331
9,hepatitis,NaN,NaN,NaN,0.760870,0.780923,0.789388,0.632950,0.695652,0.568084,NaN,NaN,0.744933,NaN,NaN,0.808696,0.797101,0.789855
